### Connecting to AWS



In [ ]:
import mysql.connector
from mysql.connector import errorcode

In [ ]:
cnx = mysql.connector .connect(
    host = 
    user = 
    passwd = 
)

In [ ]:
cursor = cnx.cursor()

### Creating the Database

In [ ]:
def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

In [ ]:
db_name = 'yelp'

In [ ]:
create_database(cursor, db_name)

In [ ]:
try:
    cursor.execute("USE {}".format(db_name))

except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

### Creating the Tables

In [ ]:
DB_NAME = 'yelp'

TABLES = {}
TABLES['business'] = (
    "CREATE TABLE business ("
    "  bus_id varchar(100),"
    "  name varchar(100),"
    "  location varchar(100),"
    "  zip_code varchar(5) NOT NULL,"
    "  price varchar(5),"
    "  rating float,"
    "  PRIMARY KEY (bus_id)"
    ") ENGINE=InnoDB")

TABLES['reviews'] = (
    "CREATE TABLE reviews ("
    "  bus_id varchar(100),"
    "  review_id varchar(100),"
    "  description varchar(100),"
    "  review_count int,"
    "  rating varchar(5) NOT NULL,"
    "  PRIMARY KEY (review_id)"
    ") ENGINE=InnoDB")


In [ ]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

        
cnx.commit()
cursor.close()
cnx.close()

### Connecting to Yelp API for Business Information

In [ ]:
import requests
import json

In [2]:
url = 'https://api.yelp.com/v3/businesses/search'
lient_id = 'v2Puv78AEi2HDDvyl9K8rg'
api_key = 'OLhy45aVXLmJtClJ75M_1KgoH85lErHLSSSSLTyyeYKb0xTey06_AdUYT3xiEa6hhkHyK-5ttHePmB2wuxIWunAzl5ykGADyj4Ct3tIxkarF14dx8bznH4C4_mHpXXYx'

In [3]:
headers = {'Authorization': 'Bearer {}'.format(api_key),}
term = 'Starbucks'
location = 'Financial District'
status = True
limit = 50  


In [ ]:
url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'open_now' : status,
                'limit' : limit
            }

### Requesting Businesses from API

In [ ]:
cursor.execute('''USE yelp''')

In [ ]:
response = requests.get(url, headers=headers, params=url_params)
data = response.json()
buz_data = parse_buz(data['businesses'])

### Parsing Business Data

In [ ]:
def parse_buz(l):
    parsed_data = []
    for business in l:
        try:
            biz_tuple = (business['id'], business['name'], ' '.join(business['location']['display_address']), business['location']['zip_code'], business['price'], business['rating'])
        except:
            biz_tuple = (business['id'], business['name'], ' '.join(business['location']['display_address']), business['location']['zip_code'], None, business['rating'])
        parsed_data.append(biz_tuple)
    return parsed_data

### Inserting Parsed Data into Business Table

In [ ]:
stmt = """INSERT INTO business (bus_id, name, location, zip_code, price, rating) VALUES (%s, %s, %s, %s, %s, %s)"""
cursor.executemany(stmt, buz_data)
cnx.commit()

### Grabing All Search Results

In [ ]:
cur = 0
while cur < 1500 and cur < 1000:
    url_params['offset'] = cur
    results = bus_call(url_params, api_key)
    parsed_results = parse_buz(results)
    db_insert(parsed_results)
    cur += 50

### Combined Function for Business Information

In [ ]:
def combined_function():
    cur = 0
    while cur < 1500 and cur < 1000:    
        url_params['offset'] = cur
        response = requests.get(url, headers=headers, params=url_params)
        data = response.json()
        buz_data = parse_buz(data['businesses'])
        stmt = """
            INSERT INTO business (bus_id, name, location, zip_code, price, rating) VALUES (%s, %s, %s, %s, %s, %s)
            """
        cursor.executemany(stmt, buz_data)
        cnx.commit()
        cur += 50

### Requesting Review Data from API

In [ ]:
url = 'https://api.yelp.com/v3/businesses/{id}/reviews'
client_id = 'v2Puv78AEi2HDDvyl9K8rg'
api_key = '-0Z90lzWTGK-FFDi95t-4bUPkgJnkWfiLGNBL2CqstzNhkgUZU_342EExhX2d-Xy1pwuXKGGX-S7qItCqTEYKnA6eNBZKGjFos_9EfbQXx0IKMtXPwa2LS1M42HpXXYx'

In [ ]:
url_params_r = {'locale' : 'en_US'}
headers = {'Authorization': 'Bearer {}'.format(api_key),}
response = requests.get(url, headers=headers, params=url_params)

### Obtaining List of Business ID's

In [ ]:
cursor.execute('''SELECT bus_id FROM yelp.business''')
bus_id = cursor.fetchall()

### Parsing Review Data

In [ ]:
def parse_rev(business,bus_id):
    res_reviews = []
    rev_tuple1 = (bus_id, business['reviews'][0]['id'], business['reviews'][0]['text'],business['total'], business['reviews'][0]['rating'])
    rev_tuple2 = (bus_id, business['reviews'][1]['id'], business['reviews'][1]['text'],business['total'], business['reviews'][1]['rating'])
    rev_tuple3 = (bus_id, business['reviews'][2]['id'], business['reviews'][2]['text'],business['total'], business['reviews'][2]['rating'])
    rev_list = [rev_tuple1, rev_tuple2, rev_tuple3]
    res_reviews.extend(rev_list)  
    return res_reviews

### Inserting Review Data into Table

In [ ]:
stmt = """INSERT INTO reviews (bus_id, review_id, description ,review_count, rating) VALUES (%s, %s, %s, %s, %s)"""
cursor.executemany(stmt, parsed_data)

### Combined Function for Review Information

In [ ]:
def rev_call(bus_id):
    review_url = f"https://api.yelp.com/v3/businesses/{bus_id}/reviews"
    response1 = requests.get(review_url, headers=headers, params=url_params)
    review = response1.json()
    parsed_data = parse_rev(review, bus_id)
    stmt = """
        INSERT INTO reviews (bus_id, review_id, description ,review_count, rating) VALUES (%s, %s, %s, %s, %s)
        """
    cursor.executemany(stmt, parsed_data)
    cnx.commit()

In [ ]:
for i in range(0,len(bus_id)):
    try:
        rev_call(bus_id[i][0])
    except:
        continue
        